<a href="https://colab.research.google.com/github/FlyPig23/CSE5526/blob/main/CSE_5526_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline System

In [ ]:
# Import standard PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# Import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms

The following library call downloads the training set and puts it into data/FashionMNIST, and prepares the dataset to be passed into a PyTorch as a tensor.

In [ ]:
# Use standard FashionMNIST dataset
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = False,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

Define a network that is a Convolutional Neural Network.

In [ ]:
# Build the neural network, expand on top of nn.Module
class Network(nn.Module):
  def __init__(self):
    super().__init__()

    # Define layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels = 5, out_channels = 10, kernel_size = 5)

    self.fc1 = nn.Linear(in_features = 4 * 4 * 10, out_features = 100)
    self.fc2 = nn.Linear(in_features = 100, out_features = 50)

    self.out = nn.Linear(in_features = 50, out_features = 10)

  # Define forward function
  def forward(self, t):
    # conv1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # conv2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # fc1
    # Use PyTorch's tensor operation t.reshape to flatten the tensor so it can 
    # be passed to the dense layer afterward
    t = t.reshape(-1, 4 * 4 * 10)
    t = self.fc1(t)
    t = F.relu(t)

    # fc2
    t = self.fc2(t)
    t = F.relu(t)

    # output
    t = self.out(t)
    # Don't need softmax here since we'll use cross-entropy as activation.

    return t

Auxiliary function that reports the accuracy on a dataset.

In [ ]:
def get_accuracy(model, dataloader):
  count = 0
  correct = 0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds = network(images)
      batch_correct = preds.argmax(dim = 1).eq(labels).sum().item()
      batch_count = len(batch[0])
      count += batch_count
      correct += batch_correct
  model.train()
  return correct/count

Train the model for ten epochs, report the training set accuracy after each epoch.

In [ ]:
learning_rate = 0.001
batch_size = 1000
shuffle = True
epochs = 10

network = Network()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr = learning_rate)

# Set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch, get_accuracy(network, loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch, get_accuracy(network, test_loader)))

Epoch 0: train set accuracy 0.6333333333333333
Epoch 1: train set accuracy 0.7264166666666667
Epoch 2: train set accuracy 0.7545
Epoch 3: train set accuracy 0.7753166666666667
Epoch 4: train set accuracy 0.7900166666666667
Epoch 5: train set accuracy 0.8000166666666667
Epoch 6: train set accuracy 0.8069166666666666
Epoch 7: train set accuracy 0.8157833333333333
Epoch 8: train set accuracy 0.8230166666666666
Epoch 9: train set accuracy 0.83105
Epoch 9: test set accuracy 0.8241


# Comparison System

## More Feature Maps (20 feature maps for layer 1, 40 feature maps for layer 2)

In [ ]:
def comparison1_get_accuracy(model, dataloader):
  count = 0
  correct = 0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds = comparison1_network(images)
      batch_correct = preds.argmax(dim = 1).eq(labels).sum().item()
      batch_count = len(batch[0])
      count += batch_count
      correct += batch_correct
  model.train()
  return correct/count

In [ ]:
# Build the neural network, expand on top of nn.Module
class Comparison1_Network(nn.Module):
  def __init__(self):
    super().__init__()

    # Define layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 20, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels = 20, out_channels = 40, kernel_size = 5)

    self.fc1 = nn.Linear(in_features = 4 * 4 * 40, out_features = 100)
    self.fc2 = nn.Linear(in_features = 100, out_features = 50)

    self.out = nn.Linear(in_features = 50, out_features = 10)

  # Define forward function
  def forward(self, t):
    # conv1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # conv2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # fc1
    # Use PyTorch's tensor operation t.reshape to flatten the tensor so it can 
    # be passed to the dense layer afterward
    t = t.reshape(-1, 4 * 4 * 40)
    t = self.fc1(t)
    t = F.relu(t)

    # fc2
    t = self.fc2(t)
    t = F.relu(t)

    # output
    t = self.out(t)
    # Don't need softmax here since we'll use cross-entropy as activation.

    return t

In [ ]:
learning_rate = 0.001
batch_size = 1000
shuffle = True
epochs = 10

comparison1_network = Comparison1_Network()
comparison1_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
comparison1_optimizer = optim.Adam(comparison1_network.parameters(), lr = learning_rate)

# Set the network to training mode
comparison1_network.train()
for epoch in range(epochs):
  for batch in comparison1_loader:
    comparison1_images = batch[0]
    comparison1_labels = batch[1]
    comparison1_preds = comparison1_network(comparison1_images)
    comparison1_loss = F.cross_entropy(comparison1_preds, comparison1_labels)

    comparison1_optimizer.zero_grad()
    comparison1_loss.backward()
    comparison1_optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch, comparison1_get_accuracy(comparison1_network, comparison1_loader)))

comparison1_test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch, comparison1_get_accuracy(comparison1_network, comparison1_test_loader)))

Epoch 0: train set accuracy 0.6963333333333334
Epoch 1: train set accuracy 0.7555
Epoch 2: train set accuracy 0.78785
Epoch 3: train set accuracy 0.81045
Epoch 4: train set accuracy 0.8259
Epoch 5: train set accuracy 0.8365333333333334
Epoch 6: train set accuracy 0.8422166666666666
Epoch 7: train set accuracy 0.8452
Epoch 8: train set accuracy 0.8535333333333334
Epoch 9: train set accuracy 0.8605
Epoch 9: test set accuracy 0.8503


## Larger layer size (13 * 13 * 5 output for layer 1, 6 * 6 * 10 output for layer 2)

In [ ]:
def comparison2_get_accuracy(model, dataloader):
  count = 0
  correct = 0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds = comparison2_network(images)
      batch_correct = preds.argmax(dim = 1).eq(labels).sum().item()
      batch_count = len(batch[0])
      count += batch_count
      correct += batch_correct
  model.train()
  return correct/count

In [ ]:
# Build the neural network, expand on top of nn.Module
class Comparison2_Network(nn.Module):
  def __init__(self):
    super().__init__()

    # Define layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 3)
    self.conv2 = nn.Conv2d(in_channels = 5, out_channels = 10, kernel_size = 2)

    self.fc1 = nn.Linear(in_features = 6 * 6 * 10, out_features = 100)
    self.fc2 = nn.Linear(in_features = 100, out_features = 50)

    self.out = nn.Linear(in_features = 50, out_features = 10)

  # Define forward function
  def forward(self, t):
    # conv1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # conv2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # fc1
    # Use PyTorch's tensor operation t.reshape to flatten the tensor so it can 
    # be passed to the dense layer afterward
    t = t.reshape(-1, 6 * 6 * 10)
    t = self.fc1(t)
    t = F.relu(t)

    # fc2
    t = self.fc2(t)
    t = F.relu(t)

    # output
    t = self.out(t)
    # Don't need softmax here since we'll use cross-entropy as activation.

    return t

In [ ]:
learning_rate = 0.001
batch_size = 1000
shuffle = True
epochs = 10

comparison2_network = Comparison2_Network()
comparison2_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
comparison2_optimizer = optim.Adam(comparison2_network.parameters(), lr = learning_rate)

# Set the network to training mode
comparison2_network.train()
for epoch in range(epochs):
  for batch in comparison2_loader:
    comparison2_images = batch[0]
    comparison2_labels = batch[1]
    comparison2_preds = comparison2_network(comparison2_images)
    comparison2_loss = F.cross_entropy(comparison2_preds, comparison2_labels)

    comparison2_optimizer.zero_grad()
    comparison2_loss.backward()
    comparison2_optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch, comparison2_get_accuracy(comparison2_network, comparison2_loader)))

comparison2_test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch, comparison2_get_accuracy(comparison2_network, comparison2_test_loader)))

Epoch 0: train set accuracy 0.6256666666666667
Epoch 1: train set accuracy 0.7633333333333333
Epoch 2: train set accuracy 0.78425
Epoch 3: train set accuracy 0.7994333333333333
Epoch 4: train set accuracy 0.8130666666666667
Epoch 5: train set accuracy 0.82455
Epoch 6: train set accuracy 0.8322666666666667
Epoch 7: train set accuracy 0.84015
Epoch 8: train set accuracy 0.8462166666666666
Epoch 9: train set accuracy 0.8512166666666666
Epoch 9: test set accuracy 0.841


## More layers (3 convolutional layers)

In [ ]:
def comparison3_get_accuracy(model, dataloader):
  count = 0
  correct = 0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds = comparison3_network(images)
      batch_correct = preds.argmax(dim = 1).eq(labels).sum().item()
      batch_count = len(batch[0])
      count += batch_count
      correct += batch_correct
  model.train()
  return correct/count

In [ ]:
# Build the neural network, expand on top of nn.Module
class Comparison3_Network(nn.Module):
  def __init__(self):
    super().__init__()

    # Define layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 3)
    self.conv2 = nn.Conv2d(in_channels = 5, out_channels = 10, kernel_size = 3)
    self.conv3 = nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = 3)

    self.fc1 = nn.Linear(in_features = 2 * 2 * 20, out_features = 100)
    self.fc2 = nn.Linear(in_features = 100, out_features = 50)

    self.out = nn.Linear(in_features = 50, out_features = 10)

  # Define forward function
  def forward(self, t):
    # conv1
    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # conv2
    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 2)

    # conv3
    t = self.conv3(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size = 2, stride = 1)

    # fc1
    # Use PyTorch's tensor operation t.reshape to flatten the tensor so it can 
    # be passed to the dense layer afterward
    t = t.reshape(-1, 2 * 2 * 20)
    t = self.fc1(t)
    t = F.relu(t)

    # fc2
    t = self.fc2(t)
    t = F.relu(t)

    # output
    t = self.out(t)
    # Don't need softmax here since we'll use cross-entropy as activation.

    return t

In [ ]:
learning_rate = 0.001
batch_size = 800
shuffle = True
epochs = 10

comparison3_network = Comparison3_Network()
comparison3_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
comparison3_optimizer = optim.Adam(comparison3_network.parameters(), lr = learning_rate)

# Set the network to training mode
comparison3_network.train()
for epoch in range(epochs):
  for batch in comparison3_loader:
    comparison3_images = batch[0]
    comparison3_labels = batch[1]
    comparison3_preds = comparison3_network(comparison3_images)
    comparison3_loss = F.cross_entropy(comparison3_preds, comparison3_labels)

    comparison3_optimizer.zero_grad()
    comparison3_loss.backward()
    comparison3_optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch, comparison3_get_accuracy(comparison3_network, comparison3_loader)))

comparison3_test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch, comparison3_get_accuracy(comparison3_network, comparison3_test_loader)))

Epoch 0: train set accuracy 0.5713
Epoch 1: train set accuracy 0.6893333333333334
Epoch 2: train set accuracy 0.7147166666666667
Epoch 3: train set accuracy 0.7368
Epoch 4: train set accuracy 0.74865
Epoch 5: train set accuracy 0.7581833333333333
Epoch 6: train set accuracy 0.7665
Epoch 7: train set accuracy 0.7722166666666667
Epoch 8: train set accuracy 0.77815
Epoch 9: train set accuracy 0.7839666666666667
Epoch 9: test set accuracy 0.7752
